# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

**Assignment**
Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
   * Excel and Vensim are only supported on Windows
   * Vensim requires the DSS version of Vensim
   * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get [NetLogo 6.3.0](https://ccl.northwestern.edu/netlogo/download.shtml)
   * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
   * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).


|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes [example scripts](https://github.com/quaquel/EMAworkbench/tree/master/ema_workbench/examples) for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel).
  * This [tutorial](https://emaworkbench.readthedocs.io/en/latest/basic_tutorial.html) also shows a simple model in Python, Vensim and Excel connected to the workbench.

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function.

In [3]:
# Some imports you may need
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel
#from ema_workbench.connectors.vensim import VensimModel

from ema_workbench import Samplers
from ema_workbench.em_framework.samplers import LHSSampler
from ema_workbench.em_framework.salib_samplers import MorrisSampler, SobolSampler

from ema_workbench.analysis.plotting import lines, Density

In [3]:
PysdModel?


In [2]:
VensimModel?

Object `VensimModel` not found.


In [4]:
# Import the Python function
from model.pred_prey import PredPrey

In [14]:
# Define uncertainties and outcomes
uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                 RealParameter('predation_rate', 0.0005, 0.003),
                 RealParameter('predator_efficiency', 0.001, 0.004),
                 RealParameter('predator_loss_rate', 0.04, 0.08)]

outcomes = [TimeSeriesOutcome('TIME', function=np.squeeze),
            TimeSeriesOutcome('predators', function=np.squeeze),
            TimeSeriesOutcome('prey', function=np.squeeze)]

# Define model objects for the different implementations

# Python model
py_model = Model('Python', function = PredPrey)
py_model.uncertainties = uncertainties
py_model.outcomes = outcomes

#Define the NetLogo model
nl_model = NetLogoModel('NetLogo', wd='./model/', model_file="PredPrey.nlogo")
nl_model.run_length = int(365 / 0.25)
nl_model.replications = 1
nl_model.uncertainties = uncertainties
nl_model.outcomes = outcomes

#Define the Excel model
excel_model = ExcelModel('Excel', wd='./model/', model_file='PredPrey.xlsx')
excel_model.default_sheet = 'Sheet1'
excel_model.uncertainties = uncertainties
excel_model.outcomes = outcomes

# PySD model
pysd_model = PysdModel('PySD', mdl_file='./model/PredPrey.mdl')
pysd_model.uncertainties = uncertainties
pysd_model.outcomes = [TimeSeriesOutcome('TIME', function=np.squeeze),
            TimeSeriesOutcome('predators', function=np.squeeze),
            TimeSeriesOutcome('prey', function=np.squeeze)]

In [15]:
nr_experiments = 10
experiments, outcomes = perform_experiments([pysd_model],
                                  nr_experiments, uncertainty_sampling=Samplers.LHS)



  0%|                                                   | 0/10 [00:00<?, ?it/s]'TIME not found as model element'
Traceback (most recent call last):
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 91, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\em_framework\model.py", line 339, in run_model
    outputs = self.run_experiment(experiment)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\connectors\

EMAError: exception in run_model
Caused by: KeyError: 'TIME not found as model element'

In [6]:
nr_experiments = 10

#Using Latin Hypercube sampling
experiments, outcomes = perform_experiments([py_model, excel_model, pysd_model],
                                  nr_experiments, uncertainty_sampling=Samplers.LHS)

 67%|████████████████████████████              | 20/30 [00:10<00:07,  1.37it/s]'TIME not found as model element'
Traceback (most recent call last):
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 91, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\em_framework\model.py", line 339, in run_model
    outputs = self.run_experiment(experiment)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Marnix\miniconda3\envs\mbdm\Lib\site-packages\ema_workbench\connectors\py

EMAError: exception in run_model
Caused by: KeyError: 'TIME not found as model element'

In [17]:
perform_experiments?